In [1]:
import numpy as np
from itertools import combinations

base_relative_abundances = [1e-4, 1e-3, 1e-2]

relative_abundances = [relative_abundance * number
                       for relative_abundance 
                       in base_relative_abundances
                       for number in (1,2,5) 
                       for repeat in range(10)]

relative_abundances += [1-sum(relative_abundances)]
frequencies = np.array(relative_abundances)

rate = 2

**WARNING:** what follows is horrible, brute-force code, that makes almost no attempt to utilize all of the symmetries available to simplify the calculations. (The products over the strains not in the droplets are simplified though.) This is NOT recommended as a way to compute these probabilities, and is only used because it was a straightforward implementation for a one-time computation and I could let it run while eating lunch or doing other work. This does not even remotely scale, is not even remotely a good algorithm, etc. Don't take inspiration from this.

In [2]:
%%time
zero_strain_probability = np.exp(-rate)

CPU times: user 19 µs, sys: 22 µs, total: 41 µs
Wall time: 51.5 µs


In [3]:
%%time
one_strain_probability = 0.

for frequency in frequencies:
    one_strain_probability += (1. - np.exp(-rate*frequency))*np.exp(-rate*(1. - frequency))

CPU times: user 1.53 ms, sys: 0 ns, total: 1.53 ms
Wall time: 1.54 ms


In [4]:
%%time
two_strain_probability = 0.

for duple in combinations(frequencies, 2):
    freq_1, freq_2 = duple
    two_strain_probability += (1. - np.exp(-rate*freq_1))*(1. - np.exp(-rate*freq_2))*np.exp(-rate*(1. - freq_1 - freq_2))

CPU times: user 87.9 ms, sys: 0 ns, total: 87.9 ms
Wall time: 87.4 ms


In [5]:
%%time
three_strain_probability = 0.

for triple in combinations(frequencies, 3):
    freq_1, freq_2, freq_3 = triple
    three_strain_probability += (1. - np.exp(-rate*freq_1))*(1. - np.exp(-rate*freq_2))*(1. - np.exp(-rate*freq_3))*np.exp(-rate*(1. - freq_1 - freq_2 - freq_3))

CPU times: user 2.62 s, sys: 0 ns, total: 2.62 s
Wall time: 2.62 s


In [6]:
three_strain_probability

0.1793916362847235

In [7]:
%%time
four_strain_probability = 0.

for quartuple in combinations(frequencies, 4):
    freq_1, freq_2, freq_3, freq_4 = quartuple
    four_strain_probability += (1. - np.exp(-rate*freq_1))*(1. - np.exp(-rate*freq_2))*(1. - np.exp(-rate*freq_3))*(1. - np.exp(-rate*freq_4))*np.exp(-rate*(1. - freq_1 - freq_2 - freq_3 - freq_4))

CPU times: user 1min 13s, sys: 0 ns, total: 1min 13s
Wall time: 1min 13s


In [8]:
four_strain_probability

0.08172909563542519

In [9]:
%%time
five_strain_probability = 0.

for quintuple in combinations(frequencies, 5):
    freq_1, freq_2, freq_3, freq_4, freq_5 = quintuple
    five_strain_probability += (1. - np.exp(-rate*freq_1))*(1. - np.exp(-rate*freq_2))*(1. - np.exp(-rate*freq_3))*(1. - np.exp(-rate*freq_4))*(1. - np.exp(-rate*freq_5))*np.exp(-rate*(1 - freq_1 - freq_2 - freq_3 - freq_4 - freq_5))

CPU times: user 25min 46s, sys: 0 ns, total: 25min 46s
Wall time: 25min 50s


In [10]:
five_strain_probability

0.028465155323010822

In [11]:
more_strains_probability = 1. - zero_strain_probability - one_strain_probability - two_strain_probability - three_strain_probability - four_strain_probability - five_strain_probability

In [12]:
import json

results = {'zero_strain':zero_strain_probability,
           'one_strain':one_strain_probability, 'two_strain': two_strain_probability,
           'three_strain': three_strain_probability, 'four_strain': four_strain_probability,
           'five_strain': five_strain_probability, 'more_strains': more_strains_probability}

with open('multi_strain_probabilities.json', 'w') as file_pointer:
    json.dump(results, file_pointer)